In [265]:
from numpy import pi
from qiskit import (Aer, ClassicalRegister, QuantumCircuit, QuantumRegister,
                    assemble, transpile)
from qiskit.visualization import plot_histogram, plot_bloch_vector


def create_QuantumRegister(size=1, name='q'):
    return QuantumRegister(size, name)


def create_ClassicalRegister(size=1, name='c'):
    return ClassicalRegister(size, name)


def generate_QuantumCircuit(quantum_register, classical_register):
    return QuantumCircuit(quantum_register, classical_register)

# Block 1: Convert integer to binary for initilization phase

In [369]:
# Integer Input

num_1 = 3
num_2 = 3

In [313]:
def binarize(num):
    return bin(num)[2:]

# Block 2: Design a circuit

In [536]:
def adder(num_1, num_2):
    qr, c, q1, q2 = create_register(num_1, num_2)
    qc = init_circuit(num_1, num_2)

    init_1 = [0 for _ in range(2**len(q1))]
    init_2 = [0 for _ in range(2**len(q1))]
    init_1[num_1] = 1
    init_2[num_2] = 1

    qc.initialize(init_1, q1)
    qc.initialize(init_2, q2)
    
    qc.h(q2)
    
    qc.z(q1)
    
    for i in range(len(q1)):
        qc.cx(q1[i], qr[i])
        qc.cx(q2[i], qr[i])
        qc.toffoli(q1[i], q2[i], qr[i + 1])

    qc.measure(qr, c)
    return qc

def simulate(qc):
    
    simulator = Aer.get_backend('aer_simulator')
    qc = transpile(qc, simulator)

    # Run and get counts
    result = simulator.run(qc).result()
    counts = result.get_counts(qc)
    plot_histogram(counts, title='Bell-State counts', number_to_keep=len(counts))
    return counts

In [537]:
qc = adder(22, 3)
counts = simulate(qc)


(QuantumRegister(6, 'qr'), ClassicalRegister(6, 'c'), QuantumRegister(5, 'q0'), QuantumRegister(5, 'q1'))


In [548]:
plot_histogram(counts, number_to_keep=len(counts))
num =  max(counts.items())[0]

In [549]:
hex('0b101111')

TypeError: 'str' object cannot be interpreted as an integer


# Modified

In [501]:
def bit_length_init(*num):
    bit_lenght = 0
    for n in num:
        bit_lenght = max(bit_lenght, len(binarize(n)))
    return bit_lenght


def create_register(*num):
    length = bit_length_init(*num)

    params = []
    params.append(QuantumRegister(length + int((len(num) + 1) / 2), 'qr'))
    params.append(ClassicalRegister(length + int((len(num) + 1) / 2), 'c'))
    for i in range(len(num)):
        params.append(QuantumRegister(length, 'q{}'.format(i)))
    return tuple(params)


def init_circuit(*num):
    params = create_register(*num)
    print(params)
    return QuantumCircuit(*params)


def adder(*num):
    qr, c, q1, q2 = create_register(*num)
    qc = init_circuit(*num)

    init_1 = [0 for _ in range(2**len(q1))]
    init_2 = [0 for _ in range(2**len(q1))]
    init_1[num[0]] = 1
    init_2[num[1]] = 1

    qc.initialize(init_1, q1)
    qc.initialize(init_2, q2)

    for i in range(len(q1)):
        qc.cx(q1[i], qr[i])
        qc.cx(q2[i], qr[i])
        qc.toffoli(q1[i], q2[i], qr[i + 1])

    qc.measure(qr, c)
    return qc

In [508]:
qc = adder(3, 2)

(QuantumRegister(3, 'qr'), ClassicalRegister(3, 'c'), QuantumRegister(2, 'q0'), QuantumRegister(2, 'q1'))


In [507]:
simulate(qc)

(Result(backend_name='aer_simulator', backend_version='0.8.0', qobj_id='c7eb19c2-dd83-4a66-81c4-96e733b397db', job_id='5144bae7-881e-4acf-9dfe-292c8fb17f83', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=MeasLevel.CLASSIFIED, data=ExperimentResultData(counts={'0x7': 1024}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2], ['c', 3]], creg_sizes=[['c', 4]], global_phase=0.0, memory_slots=4, metadata=None, n_qubits=10, name='circuit-243', qreg_sizes=[['qr', 4], ['q0', 3], ['q1', 3]], qubit_labels=[['qr', 0], ['qr', 1], ['qr', 2], ['qr', 3], ['q0', 0], ['q0', 1], ['q0', 2], ['q1', 0], ['q1', 1], ['q1', 2]]), status=DONE, seed_simulator=1449502139, metadata={'parallel_shots': 4, 'device': 'CPU', 'parallel_state_update': 1, 'measure_sampling': False, 'method': 'statevector', 'fusion': {'applied': False, 'max_fused_qubits': 5, 'enabled': True, 'threshold': 14}}, time_taken=0.11679511100000001)], date=2021-04-11T16:37:49.157828, status=COMP